## Create Datasets: RP-Mod, RP-Crowd-1-5, DerStandard

In [31]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tikzplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import textwrap
import warnings
from tqdm import tqdm

In [71]:
data = pd.read_csv('../Dataset/Text-Data/RP-Mod-Crowd.csv')
print("Number of comments: " + str(len(data)))
data.head()

Number of comments: 85000


,Unnamed: 0,Unnamed: 0.1,id,Text,Reject Newspaper,Reject Crowd,Rejection Count Crowd,Sexism Count Crowd,Racism Count Crowd,Threat Count Crowd,Insult Count Crowd,Profanity Count Crowd,Meta Count Crowd,Advertisement Count Crowd
0,0,0,1911223,Niemand braucht Laschet den Merkel Günstling !...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1911225,das war apokalypse now. nicht einmal zu einem ...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1911229,Katastrophal - Katastrophal - anders kann d...,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,3,1911239,Dann sollten wir unsere Rüstungsexporte schnel...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1911243,"na ja,im notfall sind wir amis ja noch da um z...",0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# Calculate number of comments rejected and accepted by mods in RP dataset
mod_rejected = np.sum(data["Reject Newspaper"])

mod_accepted = len(np.array(data["Reject Newspaper"])) -  mod_rejected

print("Number of comments rejected by moderator " + str(mod_rejected))
print("Number of comments accapted by moderator " + str(mod_accepted))

moderator_rejections = np.array(data[data["Reject Newspaper"]==1].Text)
moderator_acceptance = np.array(data[data["Reject Newspaper"]==0].Text)
min(data["Rejection Count Crowd"])

Number of comments rejected by moderator 7141
Number of comments accapted by moderator 77859


0

In [178]:
## we randomly undersample
np.random.seed(seed=42)
length = len(moderator_rejections)
ind = np.random.choice(len(moderator_acceptance), length, replace=False)

## we create labels
labs = np.concatenate((np.ones(length),np.zeros(length)))

textdata =list(moderator_rejections)
textdata.extend(list(moderator_acceptance[ind]))  

subset_data = pd.DataFrame({'text':textdata,'label':labs})
subset_data.to_csv("../Dataset/abusive_moderators_NEW.csv",index=False)

In [135]:
## now we create the datasets for the crowd RP-Crowd-1..5
comment_ids = []
num_rejected = []

## replace nan with 0
data = data.fillna(0)

for min_rejects in range(1,6):
    #aggregation = dat.groupby('ID').agg({'Rejected':['sum']})
    #aggregation.columns = ['Abgelehnt']
    sexismus = data['Sexism Count Crowd']
    #sexismus.columns = ['Abgelehnt']
    rassismus = data['Racism Count Crowd']
    #rassismus.columns = ['Abgelehnt']
    beleidigung = data['Insult Count Crowd']
    #beleidigung.columns = ['Abgelehnt']
    drohung = data['Threat Count Crowd']
    #drohung.columns = ['Abgelehnt']
    profane = data["Profanity Count Crowd"]
    
    
    #print(sexismus.head())
    reasons = [sexismus,rassismus,beleidigung,drohung,profane,data["Meta Count Crowd"],data["Advertisement Count Crowd"]]

    indices = []
    textdat = list()
    ids = list()

    for reason in reasons:

        index_rejected = reason.index.values[reason >= min_rejects]
        indices.extend(index_rejected)

    a = np.unique(indices)
    num_rejected.append(len(a))
    print(len(a))
    
    textdata =list(data.iloc[a,3])
    helpme = list(data.iloc[a,2])
    

    
    ids.extend(helpme)
    #print(ids)
    textdat.extend(textdata)
    
    length = len(textdat)

    ###########################


    index_no_rejection = data.index.values[data["Rejection Count Crowd"] == 0]
    
    ## now we get comments that are not rejected
    no_rejected_comments = data.iloc[index_no_rejection].Text
    indi = data.iloc[index_no_rejection].id
    
    #print(len(no_rejected_comments))
    ## we randomly undersample
    np.random.seed(seed=42)
    ind = np.random.choice(len(no_rejected_comments), length, replace=False)

    ## we create labels
    labs = np.concatenate((np.ones(length),np.zeros(length)))

    textdat.extend(no_rejected_comments.iloc[ind])
    ids.extend(indi.iloc[ind])
    
    subset_data = pd.DataFrame({'id':ids,'text':textdat,'label':labs})
    print(str(len(subset_data)))
    subset_data.to_csv("../Dataset/RP-Crowd-"+str(min_rejects)+"_NEW.csv",index=False)
    comment_ids.append(ids)

28705
28705
57410
8684
8684
17368
3151
3151
6302
987
987
1974
212
212
424


In [177]:
abusive_standard = pd.read_csv('../Dataset/Text-Data/derstandard_abusive.csv',header=0)
non_abusive_standard = pd.read_csv('../Dataset/Text-Data/derstandard_non_abusive.csv', header=0)

der_standard = pd.concat([abusive_standard,non_abusive_standard])
der_standard.to_csv("../Dataset/Text-Data/derstandard.csv",index=False)